In [1]:
try:
    import transformers, emoji, soynlp, pytorch_lightning, datasets
except:
    !pip install -U -q transformers emoji soynlp pytorch-lightning datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 44.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 81.5 MB/s eta 

In [2]:
from datasets import load_dataset

kmhas = load_dataset("jeanlee/kmhas_korean_hate_speech")
unsmile = load_dataset('smilegate-ai/kor_unsmile')

Generating train split:   0%|          | 0/78977 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8776 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21939 [00:00<?, ? examples/s]

Dataset kmhas_korean_hate_speech downloaded and prepared to /root/.cache/huggingface/datasets/jeanlee___kmhas_korean_hate_speech/default/1.0.0/17406fbed45548c92e0795df0675e21fb2a09ceaa098bd5ff58c7fdc7f8a63d4. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/15005 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/3737 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/smilegate-ai___parquet/smilegate-ai--kor_unsmile-e0f75c6e3be1af78/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
import pandas as pd
import numpy as np

combined = []

for data in kmhas:
  for row in zip(kmhas[data]['text'], kmhas[data]['label']):
    if len(row[1]) == 1 and 8 in row[1]:
      combined.append((row[0], 0))
    else:
      combined.append((row[0], 1))

for data in unsmile:
  for row in zip(unsmile[data]['문장'], unsmile[data]['clean']):
    if row[1] == 1:
      combined.append((row[0], 0))
    else:
      combined.append((row[0], 1))

dataset = pd.DataFrame(data=combined, columns=['document', 'label'])
dataset['label'].value_counts()

0    64289
1    64145
Name: label, dtype: int64

In [4]:
def train_validate_test_split(df, train_percent=.7, validate_percent=.2):
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [5]:
train_data, validate_data, test_data = train_validate_test_split(dataset)

# 패키지 import & 기본 Args 설정

In [6]:
import os
import pandas as pd

from pprint import pprint

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR

from pytorch_lightning import LightningModule, Trainer, seed_everything

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
import emoji
from soynlp.normalizer import repeat_normalize

## 기본 학습 Arguments

In [8]:
args = {
    'random_seed': 42, # Random Seed
    'pretrained_model': 'beomi/KcBERT-v2023',  # Transformers PLM name
    'pretrained_tokenizer': '',  # Optional, Transformers Tokenizer Name. Overrides `pretrained_model`
    'batch_size': 32,
    'lr': 5e-6,  # Starting Learning Rate
    'epochs': 5,  # Max Epochs
    'max_length': 256,  # Max Length input size
    'test_mode': False,  # Test Mode enables `fast_dev_run`
    'optimizer': 'AdamW',  # AdamW vs AdamP
    'lr_scheduler': 'exp',  # ExponentialLR vs CosineAnnealingWarmRestarts
    'fp16': True,  # Enable train on FP16(if GPU)
    'tpu_cores': 0,  # Enable TPU with 1 core or 8 cores
    'cpu_workers': os.cpu_count(),
}

In [9]:
args

{'random_seed': 42,
 'pretrained_model': 'beomi/KcBERT-v2023',
 'pretrained_tokenizer': '',
 'batch_size': 32,
 'lr': 5e-06,
 'epochs': 5,
 'max_length': 256,
 'test_mode': False,
 'optimizer': 'AdamW',
 'lr_scheduler': 'exp',
 'fp16': True,
 'tpu_cores': 0,
 'cpu_workers': 12}

# Model 만들기 with Pytorch Lightning

In [10]:
!nvidia-smi

Thu Jun  1 00:56:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class Model(LightningModule):
    def __init__(self, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        
        self.clsfier = AutoModelForSequenceClassification.from_pretrained(self.hparams.pretrained_model)
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.hparams.pretrained_tokenizer
            if self.hparams.pretrained_tokenizer
            else self.hparams.pretrained_model
        )
        self.training_step_outputs = []
        self.validation_step_outputs = []

    def forward(self, **kwargs):
        return self.clsfier(**kwargs)

    def training_step(self, batch, batch_idx):
        data, labels = batch
        output = self(input_ids=data, labels=labels)

        loss = output.loss
        logits = output.logits
        preds = logits.argmax(dim=-1)
        self.training_step_outputs.append({
            'loss': loss,
            'y_true': labels,
            'y_pred': preds,
        })

        return {
            'loss': loss,
            'y_true': labels,
            'y_pred': preds,
        }

    def validation_step(self, batch, batch_idx):
        data, labels = batch
        output = self(input_ids=data, labels=labels)

        loss = output.loss
        logits = output.logits
        preds = logits.argmax(dim=-1)
        self.validation_step_outputs.append({
            'loss': loss,
            'y_true': labels,
            'y_pred': preds,
        })

        return {
            'loss': loss,
            'y_true': labels,
            'y_pred': preds,
        }

    def on_train_epoch_end(self):
        outputs = self.training_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['y_true'] for x in outputs]).cpu().numpy()
        y_pred = torch.cat([x['y_pred'] for x in outputs]).cpu().numpy()

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)

        self.log('train_loss', float(loss), on_epoch=True, prog_bar=True)
        self.log('train_acc', acc, on_epoch=True, prog_bar=True)
        self.log('train_precision', prec, on_epoch=True, prog_bar=True)
        self.log('train_recall', rec, on_epoch=True, prog_bar=True)
        self.log('train_f1', f1, on_epoch=True, prog_bar=True)

        print(f'[Epoch {self.trainer.current_epoch} TRAIN] Loss: {loss}, Acc: {acc}, Prec: {prec}, Rec: {rec}, F1: {f1}')
        self.training_step_outputs.clear()  # free memory

    def on_validation_epoch_end(self):
        outputs = self.validation_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['y_true'] for x in outputs]).cpu().numpy()
        y_pred = torch.cat([x['y_pred'] for x in outputs]).cpu().numpy()

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)

        self.log('val_loss', float(loss), on_epoch=True, prog_bar=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True)
        self.log('val_precision', prec, on_epoch=True, prog_bar=True)
        self.log('val_recall', rec, on_epoch=True, prog_bar=True)
        self.log('val_f1', f1, on_epoch=True, prog_bar=True)

        print(f'[Epoch {self.trainer.current_epoch} VALIDATION] Loss: {loss}, Acc: {acc}, Prec: {prec}, Rec: {rec}, F1: {f1}')
        self.validation_step_outputs.clear()  # free memory

    def configure_optimizers(self):
        if self.hparams.optimizer == 'AdamW':
            optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)
        elif self.hparams.optimizer == 'AdamP':
            from adamp import AdamP
            optimizer = AdamP(self.parameters(), lr=self.hparams.lr)
        else:
            raise NotImplementedError('Only AdamW and AdamP is Supported!')
        if self.hparams.lr_scheduler == 'cos':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)
        elif self.hparams.lr_scheduler == 'exp':
            scheduler = ExponentialLR(optimizer, gamma=0.5)
        else:
            raise NotImplementedError('Only cos and exp lr scheduler is Supported!')
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def clean(self, x):
        emojis = ''.join(emoji.EMOJI_DATA.keys())
        pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
        url_pattern = re.compile(
            r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
        x = pattern.sub(' ', x)
        x = url_pattern.sub('', x)
        x = x.strip()
        x = repeat_normalize(x, num_repeats=2)
        return x

    def encode(self, x, **kwargs):
        return self.tokenizer.encode(
            self.clean(str(x)),
            padding='max_length',
            max_length=self.hparams.max_length,
            truncation=True,
            **kwargs,
        )

    def dataloader(self, df, shuffle=False):
        df['document'] = df['document'].map(self.encode)
        dataset = TensorDataset(
            torch.tensor(df['document'].to_list(), dtype=torch.long),
            torch.tensor(df['label'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.hparams.batch_size * 1 if not self.hparams.tpu_cores else self.hparams.tpu_cores,
            shuffle=shuffle,
            num_workers=self.hparams.cpu_workers,
        )

    def train_dataloader(self):
        return self.dataloader(train_data, shuffle=True)

    def val_dataloader(self):
        return self.dataloader(validate_data, shuffle=False)

In [12]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filename='epoch{epoch}-val_acc{val_acc:.4f}',
    monitor='val_acc',
    save_top_k=3,
    mode='max',
    auto_insert_metric_name=False,
)

# 학습

In [ ]:
!nvidia-smi

Wed May 31 07:52:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
print("Using PyTorch Ver", torch.__version__)
print("Fix Seed:", args['random_seed'])
seed_everything(args['random_seed'])
model = Model(**args)

print(":: Start Training ::")
trainer = Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=args['epochs'],
    fast_dev_run=args['test_mode'],
    num_sanity_val_steps=None if args['test_mode'] else 0,
    # For GPU Setup
    deterministic=torch.cuda.is_available(),
    accelerator="gpu",
    precision=16 if args['fp16'] and torch.cuda.is_available() else 32,
)
trainer.fit(model)

INFO:lightning_fabric.utilities.seed:Global seed set to 42


Using PyTorch Ver 2.0.1+cu118
Fix Seed: 42


Some weights of the model checkpoint at beomi/KcBERT-v2023 were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-v2023 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
Y

/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


:: Start Training ::


...

Validation: 0it [00:00, ?it/s]

[Epoch 3 VALIDATION] Loss: 0.3920589089393616, Acc: 0.8471540917231176, Prec: 0.8521580160559574, Rec: 0.8384296551184797, F1: 0.8452380952380951
[Epoch 3 TRAIN] Loss: 0.2725209891796112, Acc: 0.882184131786481, Prec: 0.8897673470312394, Rec: 0.8727640379530254, F1: 0.8811836762165437


Validation: 0it [00:00, ?it/s]

[Epoch 4 VALIDATION] Loss: 0.39434078335762024, Acc: 0.8379662072724441, Prec: 0.8148040002919921, Rec: 0.8729178071478846, F1: 0.842860379068187
[Epoch 4 TRAIN] Loss: 0.23947922885417938, Acc: 0.8990356272871873, Prec: 0.9041715417154171, Rec: 0.892940470635291, F1: 0.8985209118248796


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [14]:
model.clsfier.save_pretrained('curse_detection/kcbert-base')
model.tokenizer.save_pretrained('curse_detection/kcbert-base')

('curse_detection/kcbert-base/tokenizer_config.json',
 'curse_detection/kcbert-base/special_tokens_map.json',
 'curse_detection/kcbert-base/vocab.json',
 'curse_detection/kcbert-base/merges.txt',
 'curse_detection/kcbert-base/added_tokens.json',
 'curse_detection/kcbert-base/tokenizer.json')

In [ ]:
!cp ./lightning_logs/version_0/checkpoints/epoch4-val_acc0.8952.ckpt /curse_detection/kcelectra-base/

cp: cannot create regular file '/curse_detection/kcelectra-base/': No such file or directory


# Inference

In [ ]:
from glob import glob

latest_ckpt = sorted(glob('./lightning_logs/version_0/checkpoints/*.ckpt'))[-1]
latest_ckpt

'./lightning_logs/version_0/checkpoints/epoch0-val_acc0.9114.ckpt'

In [ ]:
model = Model.load_from_checkpoint(latest_ckpt)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.weight', 'classifier.

In [16]:
from transformers import TextClassificationPipeline

loaded_tokenizer = AutoTokenizer.from_pretrained('curse_detection/kcbert-base')
loaded_model = AutoModelForSequenceClassification.from_pretrained('curse_detection/kcbert-base', output_attentions=True)

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer, 
    model=loaded_model, 
    framework='pt',
    return_all_scores=True,
    device=0
)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [17]:
def classify_with_better_output(text_classifier, test_document):
  output = text_classifier(test_document)[0]
  print(f'origin output : {output}')
  clean = output[0]['score']
  curse = output[1]['score']
  print(f'{test_document} 가 입력되었으며,')
  if clean > curse:
    print(f'모델은 이 문장을 {clean * 100}% 확률로 깨끗한 문장이라고 추론했습니다.')
  else:
    print(f'모델은 이 문장을 {curse * 100}% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.')

In [18]:
classify_with_better_output(text_classifier, '시바견은 너무 귀엽다.')

origin output : [{'label': 'LABEL_0', 'score': 0.2566837668418884}, {'label': 'LABEL_1', 'score': 0.7433161735534668}]
시바견은 너무 귀엽다. 가 입력되었으며,
모델은 이 문장을 74.33161735534668% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.


In [19]:
classify_with_better_output(text_classifier, '시발자동차는 1955년에 출시된 우리나라 최초의 자동차이다.')

origin output : [{'label': 'LABEL_0', 'score': 0.9271190166473389}, {'label': 'LABEL_1', 'score': 0.07288097590208054}]
시발자동차는 1955년에 출시된 우리나라 최초의 자동차이다. 가 입력되었으며,
모델은 이 문장을 92.71190166473389% 확률로 깨끗한 문장이라고 추론했습니다.


In [20]:
classify_with_better_output(text_classifier, '만두 몇 개 시키고 옴')

origin output : [{'label': 'LABEL_0', 'score': 0.3961160182952881}, {'label': 'LABEL_1', 'score': 0.6038839817047119}]
만두 몇 개 시키고 옴 가 입력되었으며,
모델은 이 문장을 60.38839817047119% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.


In [21]:
classify_with_better_output(text_classifier, '아니 ㅅㅂ 네이버 메인 왜 이렇게 바꿨냐고')

origin output : [{'label': 'LABEL_0', 'score': 0.6324921250343323}, {'label': 'LABEL_1', 'score': 0.36750781536102295}]
아니 ㅅㅂ 네이버 메인 왜 이렇게 바꿨냐고 가 입력되었으며,
모델은 이 문장을 63.24921250343323% 확률로 깨끗한 문장이라고 추론했습니다.


In [22]:
labels_test = []
alpha_error = []
test = zip(test_data['document'] , test_data['label'])
for i, (doc, label) in enumerate(test):

  if i == 1000:
    break

  output = text_classifier(doc)[0]
  clean = output[0]['score']
  curse = output[1]['score']
  result = 1 if curse > clean else 0
  if (result == label):
    labels_test.append(1)
  else:
    labels_test.append(0)
  if (result != label and label == 1):
    alpha_error.append(1)
  

print(f'Accuracy Percentage with unseen data : {(sum(labels_test) / len(labels_test)) * 100}%')
print(f'Alpha Error Percentage with unseen data : {(sum(alpha_error) / sum(labels_test)) * 100}%')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Accuracy Percentage with unseen data : 77.5%
Alpha Error Percentage with unseen data : 7.225806451612904%


In [23]:
!tar -cvf 1st_kc_bert_finetuned curse_detection/
!tar -cvf 1st_kc_bert_finetuned_checkpoint lightning_logs/

curse_detection/
curse_detection/kcbert-base/
curse_detection/kcbert-base/config.json
curse_detection/kcbert-base/pytorch_model.bin
curse_detection/kcbert-base/tokenizer.json
curse_detection/kcbert-base/special_tokens_map.json
curse_detection/kcbert-base/vocab.json
curse_detection/kcbert-base/merges.txt
curse_detection/kcbert-base/tokenizer_config.json
lightning_logs/
lightning_logs/version_0/
lightning_logs/version_0/hparams.yaml
lightning_logs/version_0/events.out.tfevents.1685581036.bd86153bb72d.1889.0
lightning_logs/version_0/checkpoints/
lightning_logs/version_0/checkpoints/epoch4-val_acc0.8380.ckpt
lightning_logs/version_0/checkpoints/epoch1-val_acc0.8418.ckpt
lightning_logs/version_0/checkpoints/epoch3-val_acc0.8472.ckpt


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
!cp -r ./1st_kc_bert_finetuned ./drive/MyDrive/

In [26]:
!cp -r 1st_kc_bert_finetuned_checkpoint ./drive/MyDrive/